In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from statsmodels.formula.api import ols


In [3]:

# Set random seed for reproducibility
np.random.seed(123)

# Generate sample data
n = 1000
treatment = np.random.binomial(1, 0.5, n)
time = np.random.binomial(1, 0.5, n)
X = np.column_stack((np.ones(n), treatment, time, treatment*time))
true_betas = [1, 2, 3, 4]
y = X @ true_betas + np.random.laplace(0, 1, n)

# Create DataFrame
df = pd.DataFrame({
    'y': y,
    'treatment': treatment,
    'time': time,
    'interaction': treatment * time
})


In [4]:

# OLS estimation
ols_model = ols('y ~ treatment + time + interaction', data=df).fit()
print("OLS Estimates:")
print(ols_model.params)


OLS Estimates:
Intercept      0.974296
treatment      2.136684
time           2.829612
interaction    3.886474
dtype: float64


In [5]:

# Maximum Entropy estimation
def neg_entropy(p):
    return np.sum(p * np.log(p))

def me_objective(params):
    beta0, beta1, beta2, beta3, sigma = params
    residuals = y - X @ [beta0, beta1, beta2, beta3]
    p = np.exp(-residuals**2 / (2*sigma**2)) / np.sqrt(2*np.pi*sigma**2)
    return neg_entropy(p)

def me_constraints(params):
    beta0, beta1, beta2, beta3, sigma = params
    residuals = y - X @ [beta0, beta1, beta2, beta3]
    return [np.mean(residuals), np.mean(residuals**2) - sigma**2]


In [6]:

initial_guess = ols_model.params.tolist() + [1]
result = minimize(me_objective, initial_guess, method='SLSQP', constraints={'type': 'eq', 'fun': me_constraints})

print("\nMaximum Entropy Estimates:")
print(result.x[:4])  # First four parameters are the betas

print("\nTrue betas:")
print(true_betas)


Maximum Entropy Estimates:
[0.90349661 2.22515116 2.9426729  3.76047904]

True betas:
[1, 2, 3, 4]
